In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [3]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [9]:
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [0]:
import datetime
import glob
import os
import re

import torch
import numpy as np
from skimage import transform
from torch.utils.data.sampler import SubsetRandomSampler



class Rescale(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        normal, blurry = sample['normal'], sample['blurry']

        h, w = normal.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(normal, (new_h, new_w))
        blurry = transform.resize(blurry, (new_h, new_w))

        return {'normal': img, 'blurry': blurry}


class RandomCrop(object):

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        normal, blurry = sample['normal'], sample['blurry']

        h, w = normal.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        normal = normal[top: top + new_h,
                 left: left + new_w]
        blurry = blurry[top: top + new_h,
                 left: left + new_w]

        return {'normal': normal, 'blurry': blurry}


class ToTensor(object):
    """Convert nd-arrays in sample to Tensors."""

    def __call__(self, sample):
        normal, blurry = sample['normal'], sample['blurry']

        # swap color axis because
        # numpy normal: H x W x C
        # torch normal: C X H X W
        normal = normal.transpose((2, 0, 1))
        return {'normal': torch.from_numpy(normal),
                'blurry': torch.from_numpy(blurry)}


def findLastCheckpoint(save_dir):
    file_list = glob.glob(os.path.join(save_dir, 'model_*.pth'))
    if file_list:
        epochs_exist = []
        for file_ in file_list:
            result = re.findall(".*model_(.*).pth.*", file_)
            epochs_exist.append(int(result[0]))
        initial_epoch = max(epochs_exist)
    else:
        initial_epoch = 0
    return initial_epoch


def log(*args, **kwargs):
    print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"), *args, **kwargs)


def prepareLoaders(dataset, shuffle_dataset=True, batch_size=4, random_seed=42, validation_split=.85):
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))

    if shuffle_dataset:
        np.random.seed(random_seed)
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # building dataset slicers
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    # building dataset loaders
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                               sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)
    return train_loader, validation_loader


In [0]:
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import glob
import os




class imageDataset(Dataset):

    def __init__(self, normal_dir, blurry_dir, transform=None):
        working_dir = os.path.dirname(os.path.realpath(__file__))

        self.path_normal_dir = os.path.join(working_dir, normal_dir)
        self.path_blurry_dir = os.path.join(working_dir, blurry_dir)

        self.blurry_images = [x.replace(self.path_blurry_dir, '') for x in glob.glob(self.path_blurry_dir + '/*.jpg')]
        self.normal_images = [x.replace(self.path_normal_dir, '') for x in glob.glob(self.path_normal_dir + '/*.jpg')]

        self.transform = transform

        if self.blurry_images not in self.normal_images and len(self.blurry_images) != len(self.normal_images):
            raise Exception('mismatch between the normal images and the blurry ones')

    def __len__(self):
        return len(self.normal_images)

    def __getitem__(self, idx):
        normal_image = io.imread(self.path_normal_dir + '/' + self.normal_images[idx])
        blurry_image = io.imread(self.path_blurry_dir + '/' + self.normal_images[idx])

        sample = {'normal': normal_image, 'blurry': blurry_image}

        if self.transform:
            n, b = self.transform(sample['normal']), self.transform(sample['blurry'])
            sample = {'normal': n, 'blurry': b}

        return sample


In [0]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.utils.data
import torch.nn.init as init


# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self, depth=17, n_channels=64, image_channels=3):
        super(ConvAutoencoder, self).__init__()
        kernel_size = 3
        padding = 1
        layers = []

        layers.append(nn.Conv2d(in_channels=image_channels, out_channels=n_channels,
                                kernel_size=kernel_size, padding=padding, bias=True)
                      )
        layers.append(nn.ReLU(inplace=True))
        for _ in range(depth-2):
            layers.append(nn.Conv2d(in_channels=n_channels, out_channels=n_channels,
                                    kernel_size=kernel_size, padding=padding, bias=False
                                    )
                          )
            layers.append(nn.BatchNorm2d(n_channels, eps=0.0001, momentum=0.95))
            layers.append(nn.ReLU(inplace=True))

        layers.append(nn.Conv2d(in_channels=n_channels, out_channels=image_channels,
                                kernel_size=kernel_size, padding=padding, bias=False
                                )
                      )
        self.autoEncoder = nn.Sequential(*layers)

        self._initialize_weights()

    def forward(self, x):
        y = x
        out = self.autoEncoder(x)
        return y - out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                print('init weight')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)


In [17]:
import argparse
import os
import time

import torch.nn as nn
import torch.utils.data
from torchvision import transforms



parser = argparse.ArgumentParser(description='PyTorch ConAutoEncoder')
parser.add_argument('--model', default='ConAutoEncoder', type=str, help='choose a type of model')
parser.add_argument('--batch_size', default=2, type=int, help='batch size')
parser.add_argument('--normal_data', default='data/normal_images', type=str, help='path of train data')
parser.add_argument('--blurry_data', default='data/blurry_images', type=str, help='path of train data')
parser.add_argument('--epoch', default=5, type=int, help='number of train epoches')
parser.add_argument('--lr', default=1e-3, type=float, help='initial learning rate for Adam')
args = parser.parse_args()

data_transform = transforms.Compose([transforms.ToTensor()])
dataset = DataSet.imageDataset(args.normal_data, args.blurry_data, transform=data_transform)

train_loader, validation_loader = prepareLoaders(dataset, shuffle_dataset=True, batch_size=args.batch_size, )

save_dir = os.path.join('models', args.model)

if not os.path.exists(save_dir):
    os.mkdir(save_dir)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# initialize the NN
model = ConvAutoencoder().to(device)
# specify loss function
criterion = nn.MSELoss()

# specify loss function
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

# number of epochs to train the model
n_epochs = args.epoch

for epoch in range(1, n_epochs + 1):
    # monitor training loss
    train_loss = 0.0

    start_time = time.time()

    # train the model #
    for n_count, data in enumerate(train_loader):

        normal, blurry = data['normal'].to(device), data['blurry'].to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(blurry)
        # calculate the loss
        loss = criterion(outputs, normal)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()
        if n_count % 10 == 0:
            print('%4d %4d / %4d loss = %2.4f' % (epoch , n_count, len(train_loader), loss.item() / args.batch_size))

    elapsed_time = time.time() - start_time
    # print avg training statistics
    train_loss = train_loss / len(train_loader)
    log('epoch = %4d , loss = %4.4f , time = %4.2f s' % (epoch, train_loss, elapsed_time))

    torch.save(model, os.path.join(save_dir, 'model_%03d.pth' % epoch))





usage: ipykernel_launcher.py [-h] [--model MODEL] [--batch_size BATCH_SIZE]
                             [--normal_data NORMAL_DATA]
                             [--blurry_data BLURRY_DATA] [--epoch EPOCH]
                             [--lr LR]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-e974e967-8210-4cb3-a01b-69c87d97918b.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


python3: can't open file 'ipykernel_launcher.py': [Errno 2] No such file or directory
